In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import asyncio
import re
import time
import os

from IPython.display import display, clear_output
from novelai_api.NovelAI_API import NovelAIAPI
from novelai_api.Preset import Model, Preset
from novelai_api.GlobalSettings import GlobalSettings
from novelai_api.Tokenizer import Tokenizer
from novelai_api.utils import b64_to_tokens
from novelai_api.BiasGroup import BiasGroup

import pandas as pd

In [4]:
# Settings

# Set method for authentication
# Posible options are:
# "enter_key" (enter your access key)
# "enter_login" (enter your username & pw)
# "enter_token" (enter your access token)
# "env_key" (read access key from environment variable NAI_KEY)
# "env_login" (read user from environment variable NAI_USERNAME and password from NAI_PASSWORD)
# "env_token" (read access token from environment variable NAI_TOKEN)
auth_method = "env_token"
delay_time = 2
generation_timeout = 60
max_failed_gens = 3
candidates_goal = 10
bias_strength_inc = -0.1
bias_phrase_dict = {}
model = Model.Clio
# Preset can either be a string to use an official preset of the same name
# or a preset object to use a custom preset.
preset = Preset(name="Default", model=Model.Clio, settings={
    "stop_sequences": [],
    "temperature": 1.0,
    "max_length": 40,
    "min_length": 1,
    "top_k": 0,
    "top_a": 1.0,
    "top_p": 0.0,
    "typical_p": 0.0,
    "tail_free_sampling": 1.0,
    "repetition_penalty": 1.0,
    "repetition_penalty_range": 0,
    "repetition_penalty_slope": 0.0,
    "repetition_penalty_frequency": 0.0,
    "repetition_penalty_presence": 0.0,
    "repetition_penalty_whitelist": [],
    "repetition_penalty_default_whitelist": False,
    "length_penalty": 1.0,
    "diversity_penalty": 0.0,
    "order": [0],
    "phrase_rep_pen": "off",
})

In [5]:
auth = False
env = os.environ

# Init variable for login method
if auth_method == "enter_key":
    auth = input("Enter your NovelAI access key: ")
if auth_method == "enter_token":
    auth = input("Enter your NovelAI access token: ")
elif auth_method == "enter_login":
    auth = {}
    auth["user"] = input("Enter your NovelAI username: ")
    auth["pw"] = input("Enter your NovelAI password: ")
elif auth_method == "env_key":
    auth = env["NAI_KEY"]
elif auth_method == "env_token":
    auth = env["NAI_TOKEN"]
elif auth_method == "env_login":
    auth = {}
    auth["user"] = env["NAI_USERNAME"]
    auth["pw"] = env["NAI_PASSWORD"]
else:
    raise RuntimeError(
        "Invalid value for 'auth_method'. Must be one of 'enter_key', 'enter_token', 'enter_login', env_key', 'env_token' or 'env_login"
    )

In [6]:
async def nai_login(api, auth_method, auth):
    if auth_method == "enter_key" or auth_method == "env_key":
        await api.high_level.login_from_key(auth)
    elif auth_method == "enter_token" or auth_method == "env_token":
        await api.high_level.login_with_token(auth)
    elif auth_method == "enter_login" or auth_method == "env_login":
        await api.high_level.login(auth["user"], auth["pw"])

In [7]:
async def gen_attg_candidate(
    model=Model.Clio,
    preset = "Edgewise",
    prompt="[ Genre:",
    stop_sequences=[",", ";", " ]"],
    cut_stop_seq=True,
    auth_method="env_token",
    auth=None,
    bias_groups=None,
):
    # Initialize the NovelAI API
    api = NovelAIAPI()

    try:
        # Ensure you're logged in
        await nai_login(api, auth_method, auth)

        # If preset is a string, get the official preset with that name for the specified model
        if isinstance(preset, str):
            preset = Preset.from_official(model, preset)

        # Tokenize the stop sequences and set them for the preset
        stop_sequences_tokenized = [
            Tokenizer.encode(model, seq) for seq in stop_sequences
        ]
        preset["stop_sequences"] = stop_sequences_tokenized

        # Create default global settings
        global_settings = GlobalSettings()

        gen = await api.high_level.generate(
            prompt, model, preset, global_settings, None, bias_groups, None
        )

        # After generating the text, remove the stop sequence
        generated_text = Tokenizer.decode(model, b64_to_tokens(gen["output"]))
        if cut_stop_seq:
            for seq in stop_sequences:
                generated_text = re.sub(
                    re.escape(seq) + "$", "", generated_text
                ).strip()

        return generated_text

    except Exception as e:
        raise Exception(f"Error generating text: {e}")


def update_bias_groups(phrase, bias_phrase_dict, bias_strength_inc, bias_groups):
    # Update the bias strength for the phrase or add it if it's not in the dict
    if phrase in bias_phrase_dict:
        bias_phrase_dict[phrase] += bias_strength_inc
    else:
        bias_phrase_dict[phrase] = bias_strength_inc

    # Clear the existing bias groups
    bias_groups.clear()

    # Regenerate the bias groups based on the updated bias_phrase_dict
    for phrase, strength in bias_phrase_dict.items():
        bg = BiasGroup(strength)
        bg.add(phrase)
        bias_groups.append(bg)

In [10]:
# Initialize an empty DataFrame
df = pd.DataFrame(columns=["phrase", "count", "last_bias"])

# Counter for total generations
total_generations = 0

# set phrase biases
bias_groups = []
for phrase, strength in bias_phrase_dict.items():
    bg = BiasGroup(strength)
    bg.add(phrase)
    bias_groups.append(bg)

# Counter for unsuccessful generation attempts
unsuccessful_attempts = 0

# Loop until you have candidate_goal unique phrases
while len(df) < candidates_goal:
    total_generations += 1

    ## Clear the previous output
    clear_output(wait=True)

    print(
        f"Gen {total_generations}: Trying to gen phrase {len(df)+1}/{candidates_goal}..."
    )
    # print(f"Current bias groups {bias_phrase_dict}")

    try:
        phrase = await asyncio.wait_for(
            gen_attg_candidate(
                model=Model.Clio,
                preset=preset,
                prompt="[ Genre:",
                auth_method=auth_method,
                auth=auth,
                bias_groups=bias_groups,
            ),
            timeout=generation_timeout,
        )

        # Check if the phrase is already in the DataFrame
        if phrase in df["phrase"].values:
            df.loc[df["phrase"] == phrase, "count"] += 1
            df.loc[df["phrase"] == phrase, "last_bias"] = bias_phrase_dict.get(
                phrase, 0
            )
            print(
                f"Phrase '{phrase}' already exists. Incrementing count and changing bias by {bias_strength_inc}."
            )

            # Update the bias groups since the phrase was generated again
            update_bias_groups(phrase, bias_phrase_dict, bias_strength_inc, bias_groups)
        else:
            df.loc[len(df)] = [
                phrase,
                1,
                bias_phrase_dict.get(phrase, 0),
            ]
            print(f"Added new phrase: '{phrase}'")

        # Reset the unsuccessful_attempts counter if generation was successful
        unsuccessful_attempts = 0

    except asyncio.TimeoutError:
        print("Generation took too long. Retrying...")
        unsuccessful_attempts += 1
        if unsuccessful_attempts >= 3:
            print("3 unsuccessful generation attempts. Aborting candidate search.")
            break
    except Exception as e:
        if "Anonymous quota reached" in str(e):
            print(f"Error: {e}")
            print(
                "Anonymous rate limit reached. This indicates you are not properly authenticated. Check your authentication method. Aborting candidate search."
            )
            break
        else:
            print(f"Error: {e}")
            # import traceback
            # traceback.print_exc()
            print("Aborting candidate search")
            break

    # Wait for delay_time seconds before the next generation attempt
    time.sleep(delay_time)

print("\nCandidate search complete!")
print(df)

Gen 12: Trying to gen phrase 10/10...
Added new phrase: 'Action'

Candidate search complete!
               phrase  count  last_bias
0  Historical fiction      1        0.0
1        Contemporary      1        0.0
2           Adventure      2       -0.3
3               Lemon      1        0.0
4              LitRPG      2       -1.2
5       urban fantasy      1        0.0
6     science fiction      1        0.0
7             Fantasy      1       -0.3
8               Drama      1        0.0
9              Action      1       -0.1
